#  Initialisation de la base de donnée

Ce notebook a pour objectif de prendre la base de donnée et les metadatas, de les traiter pour noter les articles. Le but est de sauvegarder deux csv, un pour ceux avec texte complet et l'autre non. Ils contiennent des notes pour la date ainsi que les titres et abstracts transformés. Celui avec les textes complets contient aussi le nombre de fois où chaque article est cité.

Importation des dépendances

In [1]:
import numpy as np
import pandas as pd
import glob
import json
import re
import string
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import nltk

In [2]:
# à faire si besoin
#nltk.download('wordnet') 

In [3]:
root_path = '/home/arnaud/Documents/Mines1a/Informatique/Projet_informatique/data/CORD-19-research-challenge'
metadata_path = f'{root_path}/metadata.csv'
meta_df = pd.read_csv(metadata_path)

/home/arnaud/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3057: DtypeWarning: Columns (5,13,14,16) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
meta_df[0:1000].to_csv("metadata_reduced.csv")

In [5]:
def open_article(path):
    file=open(root_path+"/"+path,'r')
    article=json.load(file)
    file.close()
    return article

In [6]:
def make_text(article):
    text_list = []
    for entry in article['body_text']:
        text_list.append(entry['text'])
        text_list.append("\n")
    
    text_full=''.join(text_list)
    return text_full

In [7]:
meta_df.columns

Index(['cord_uid', 'sha', 'source_x', 'title', 'doi', 'pmcid', 'pubmed_id',
       'license', 'abstract', 'publish_time', 'authors', 'journal', 'mag_id',
       'who_covidence_id', 'arxiv_id', 'pdf_json_files', 'pmc_json_files',
       'url', 's2_id'],
      dtype='object')

## Date

In [8]:
def date(x):
    try:
        return int(x[:4])
    except TypeError:
        return np.NaN

def critère_date(date):
    try:
        return 1/(2020-date+1)
    except ZeroDivisionError:
        return 0

In [9]:
meta_df["date"]=meta_df["publish_time"].apply(lambda x: date(x))
meta_df["note_date"]=meta_df["date"].apply(lambda x:critère_date(x))

## Transformation titre et abastract

In [10]:
lemmatizer=WordNetLemmatizer()
stop=stopwords.words('english')

def preprocessing(x): #tokenisation,stop-words,lemmatization ...
    clean = re.sub(r'['+string.punctuation + '’—”'+']', "", x.lower())
    clean_text= re.sub(r'\W+', ' ', clean)
    a=""
    for i in clean_text.split():
        if i not in stop :
            a+=str(lemmatizer.lemmatize(i))+' '
    return a

def preprocessing_title(x): #tokenisation,stop-words,lemmatization ...
    clean = re.sub(r'['+string.punctuation + '’—”'+']', "", x.lower())
    clean_text= re.sub(r'\W+', ' ', clean)
    a=""
    for i in clean_text.split():
        a+=str(lemmatizer.lemmatize(i))+' '
    return a

In [11]:
meta_df["title_process"]=meta_df["title"].apply(lambda x: preprocessing_title(str(x)))
meta_df["abstract_process"]=meta_df["abstract"].apply(lambda x: preprocessing(str(x)))

## Nombre de citations

In [12]:
def has_full_text(x):
    if x["pdf_json_files"]==True and x["pmc_json_files"]==True:
        return False
    return True

In [13]:
meta_bool=meta_df[["pdf_json_files","pmc_json_files"]].isnull()
meta_bool["has_full"]=meta_bool.apply(lambda x:has_full_text(x),axis=1)
meta_df_full_text=meta_df.loc[meta_bool["has_full"]]

In [14]:
meta_df_without_text=meta_df.loc[meta_bool["has_full"]==False]

In [15]:
def get_path(path):
    a=""
    for i in range (len(path)):
        if path[i]==";":
            break
        a+=path[i]
    return a

def add_ref(article):
    ref=article["bib_entries"]

    for i in ref.keys():
        title=preprocessing_title(ref[i]["title"])
        if title in set_ref:
            dico_ref[title]+=1
        else:
            dico_ref[title]=1
            set_ref.add(title)
            
def count_ref(x):
    try:
        path=x["pdf_json_files"]
        article=open_article(get_path(str(path)))
       
    except FileNotFoundError:
        path=x["pmc_json_files"]
        article=open_article(get_path(str(path)))
    add_ref(article)
    return "done"

def link_ref(x):
    title=x
    if title in set_ref:
        return dico_ref[title]
    return 0

def path(x):
    try:
        try :
            path=x["pdf_json_files"]
            article=open_article(get_path(str(path)))
            return get_path(str(path))
       
        except FileNotFoundError:
            path=x["pmc_json_files"]
            article=open_article(get_path(str(path)))
            return get_path(str(path))
    except FileNotFoundError:
        return ""

In [16]:
dico_ref=dict()
set_ref=set()
meta_df_full_text["ref"]=meta_df_full_text[["pdf_json_files","pmc_json_files"]].apply(lambda x: count_ref(x),axis=1)

/home/arnaud/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [17]:
meta_df["nb_ref_linked"]=meta_df["title_process"].apply(lambda x: link_ref(x))

In [18]:
max_ref=meta_df["nb_ref_linked"].max()
meta_df["note_ref"]=meta_df["nb_ref_linked"].apply(lambda x:x/max_ref)
meta_df["path"]=meta_df[["pdf_json_files","pmc_json_files"]].apply(lambda x:path(x),axis=1)

In [19]:
meta_df_save=meta_df[["title","abstract","authors","title_process","abstract_process","date","path","note_date","nb_ref_linked","note_ref"]]


In [20]:
meta_df_save.to_csv("metadata_processed.csv")
